# Library

In [ ]:
import warnings

warnings.filterwarnings("ignore", message="Boolean Series key will be reindexed to match DataFrame index")
warnings.filterwarnings("ignore", category=FutureWarning, module="pyntcloud.io.ply")
warnings.filterwarnings("ignore", category=RuntimeWarning)

import os
import time
import itertools
import shutil
import subprocess

import pandas as pd
import numpy as np

from scipy.spatial import ConvexHull
from pyntcloud import PyntCloud

from vggsfm.datasets.imc_helper import (
        read_cameras_binary, write_cameras_text,
        read_images_binary, write_images_text,
        read_points3D_binary, write_points3D_text
    )

from Utilities.utilities import read_points3D, write_ply, calculate_volume, calculate_volume_file

# Calculate Volume

In [ ]:
def get_folders_in_directory(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    return folders

def list_directory_contents(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    return items

def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Get the list of folders in the directory
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3'
folders = get_folders_in_directory(directory_path)
folders.sort()

# print(folders)

##################################################
# List Algorithm
##################################################
list_algorithm = [
    'MVG',
    'MVG_MVS',
    'COLMAP',
    'COLMAP_MVS',
    # 'VGGSFM', # Use Python 3.12.7
    # 'MVE',
]

##################################################
# Run No Background
##################################################
list_bg = [
    'with_bg', 
    # 'no_bg',
    ]

##################################################
# Scale and Frame
##################################################
# list_scale_percent = [100, 50]
# list_scale_percent = [50]
list_scale_percent = [50]
# list_max_frames = [30, 45, 60, 75, 90, 105]
list_max_frames = [3]


# list_subject_testing = ['8', '9', '17', '19', '22', '67']
# list_subject_testing = [1, 2, 6, 7, 8, 9, 
#                         10, 11, 12, 16, 17, 18, 19, 
#                         21, 22, 23, 24, 27, 29, 
#                         30, 35, 
#                         41, 45, 46, 49, 
#                         51, 55, 56, 59, 
#                         60, 61, 63, 67, 69]
# list_subject_testing = [str(x) for x in list_subject_testing]

##################################################
# Run
##################################################
results_volume = []


for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    for (algorithm, bg) in itertools.product(list_algorithm, list_bg):
        # folders = ['2']
        for folder in folders:
            folder_path = os.path.join(directory_path, folder)
            
            # Get the list of files and folders in the directory
            contents = list_directory_contents(folder_path)
            contents = [x for x in contents if ':Zone.Identifier' not in x]
            subject = folder_path.split('/')[-1].split('.')[0]

            # if subject in list_subject_testing:
            if True:

                for data in contents:         
                    print('')       
                    print(f'Subject {subject} - {data} - {bg} - {algorithm} - Scale: {scale_percent}, Max Frames: {max_frames}')
                    
                    # If no bg
                    if bg == 'with_bg':
                        path_bg = ''
                    else:
                        path_bg = '_no_bg'
                        
                    start_time = time.time()
                    error = ''
                    
                    path_base = f'/home/weiyanpeh/Git/SFM_Related/CADENCE'

                    if "COLMAP" in algorithm:
                        path_volume = f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm/dense/sparse"
                        points3D = read_points3D_binary(path_volume + "/points3D.bin")

                        write_points3D_text(points3D, path_volume + "/points3D.txt")

                        points3D_file = path_volume + "/points3D.txt"
                        output_ply_file = path_volume + "/cloud_and_poses.ply"
                        
                        points3D = read_points3D(points3D_file)
                        write_ply(points3D, output_ply_file)

                        print(f'PLY file saved to {output_ply_file}')


                    dict_algorithm_path = {
                        'MVG': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/MVG_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm", 
                            'file_name': ["cloud_and_poses.ply"]},
                        
                        'MVG_MVS': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/MVG_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/mvs", 
                            'file_name': ["scene_dense_mesh.ply", "scene_dense_texture.ply", "scene_dense_mesh_refine.ply"]},
                            # "scene_dense.ply" always get error "field 'n_coords' occurs more than once"
                        
                        'COLMAP_MVS': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/mvs", 
                            'file_name': ["scene_dense_mesh.ply", "scene_dense_texture.ply", "scene_dense_mesh_refine.ply"]},
                            # "scene_dense.ply" always get error "field 'n_coords' occurs more than once"

                        # OpenMVS Colamp
                        'COLMAP': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm/dense/sparse", 
                            'file_name': ["cloud_and_poses.ply"]},
                        
                        # 'MVG': {
                        #     'file_path': f"{path_base}/SHAPE_Frames_output/MVG/{subject}/{data}/reconstruction_sequential",
                        #     'file_name': ["cloud_and_poses.ply"]},
                        
                        # Real COLMAP
                        # 'COLMAP': {
                            # 'file_path': f"{path_base}/SHAPE_Frames_input/images_input_colmap_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/dense", 
                            # 'file_name': ["fused.ply", "meshed-poisson.ply", "meshed-delaunay.ply"]},
                        
                        'VGGSFM': {
                            'file_path': f"{path_base}/SHAPE_Frames_input/images_input_vggsfm_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sparse", 
                            'file_name': ["output.ply"]},

                        'MVE': {
                            'file_path': f"{path_base}/SHAPE_Frames_input/images_input_mve_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/convert", 
                            'file_name': ["pset-L2.ply", "surface-L2.ply", "surface-L2-clean.ply"]},

                    }

                    file_path = dict_algorithm_path[algorithm]['file_path']
                    list_file_name = dict_algorithm_path[algorithm]['file_name']
                    sfm_model = ''
                    
                    if algorithm in ['MVG', 'MVG_MVS']:
                        sfm_model = 'MVG_MVS'
                    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
                        sfm_model = 'COLMAP_MVS'
                    else:
                        sfm_model = algorithm

                    for file_name in list_file_name:
                        error = ''
                        try:
                            start_time = time.time()  # Record the start time

                            print(file_path)
                            print(file_name)

                            volume = calculate_volume_file(
                                file_path = file_path, 
                                file_name = file_name
                                )
                            
                            print(f'{file_name} - Volume: {round(volume, 3)} unit3')
                            
                            end_time = time.time()  # Record the end time
                            execution_time = end_time - start_time  # Calculate the execution 
                            
                            results_volume.append([
                                subject, data, sfm_model,
                                algorithm, bg, scale_percent, max_frames,
                                execution_time, 'PyntCloud', file_path, file_name, 
                                volume, error])
                            
                        except Exception as error:
                            print(file_name, error)
                            end_time = time.time()  # Record the end time
                            execution_time = end_time - start_time  # Calculate the execution 
                            results_volume.append([
                                subject, data, sfm_model,
                                algorithm, bg, scale_percent, max_frames, 
                                execution_time, 'PyntCloud', file_path, file_name, 
                                0, error])

# Convert to DataFrame
columns = ['Subject', 'Image Data', 'SFM',
           'Algorithm', 'BG', 'Scale', 'Max Frames', 
           'Execution_time', 'Volume Calculation', 'File_Path', 'File_Name', 
           'Volume', 'Error']
df_results = pd.DataFrame(results_volume, columns=columns)
df_results = df_results.drop('Execution_time', axis=1)
df_results.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume.csv', index=False)
    


In [ ]:
df_results

In [ ]:
df_results_grouped = df_results.copy()

df_results_grouped = df_results_grouped[df_results_grouped['Volume'] != 0].reset_index(drop=True)
df_results_grouped = df_results_grouped.drop(['BG', 'Scale', 'Max Frames'], axis=1)

# Group by Subject, Image Data, and Algorithm
df_results_grouped = df_results_grouped.groupby(['Subject', 'Image Data', 'Algorithm', 'File_Name']).sum()

# Check if Volume is non-zero
df_results_grouped['!0'] = df_results_grouped['Volume'] != 0
df_results_grouped.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume_grouped.csv')

print(df_results_grouped)

In [ ]:
len(set(list(df_results['Image Data'].tolist())))

In [ ]:
list_unique_images = list(set(df_results['Image Data'].tolist()))
print('Unique Images:', len(list_unique_images))
n = list_unique_images

list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']

for algorithm in list_algorithm:
    df_temp = df_results[(df_results['Algorithm'] == algorithm)].reset_index(drop=True)
    df_temp = df_temp[df_temp['Volume'] != 0].reset_index(drop=True)
    
    if algorithm in ['MVG', 'COLMAP']:
        df_temp = df_temp[(df_temp['File_Name'] == 'cloud_and_poses.ply')].reset_index(drop=True)
        
        print('')
        print('Algorithm:', algorithm)
        print('Total number of videos:', len(df_temp))
        
    else:
        list_mesh = ['scene_dense_mesh.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_texture.ply']
        
        for mesh in list_mesh:
            df_temp_mesh = df_temp[(df_temp['File_Name'] == mesh)].reset_index(drop=True)
            
            print('')
            print('Algorithm:', algorithm, '-', mesh)
            print('Total number of videos:', len(df_temp_mesh))

In [ ]:
list_subject_testing = list(set(df_results['Subject'].tolist()))
list_subject_testing = [int(x) for x in list_subject_testing]
list_subject_testing.sort()
list_subject_testing = [str(x) for x in list_subject_testing]
# list_subject_testing = ['8', '9', '17', '19', '22', '67']

# list_background = ['with_bg', 'no_bg']
# list_scale_percent = [100, 50]

list_background = ['with_bg']
list_scale_percent = [50]
list_max_frames = [3]
df_results_save = []



list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']

for algorithm in list_algorithm:

    if algorithm in ['MVG', 'COLMAP']:
        list_mesh = ['cloud_and_poses.ply']
    else:
        list_mesh = ['scene_dense_mesh.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_texture.ply']
    
    if algorithm in ['MVG', 'MVG_MVS']:
        sfm_model = 'MVG_MVS'
    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
        sfm_model = 'COLMAP_MVS'
    else:
        sfm_model = algorithm
    
    for file_name in list_mesh:
    
        df_test = df_results[
            (df_results['Algorithm'] == algorithm) &
            (df_results['File_Name'] == file_name)]
        
        total = len(df_test)
        fail = len(df_test[df_test['Volume'] == 0])
        success = total - fail
        try:
            success_rate = round(success/total*100, 1)
        except:
            success_rate = 0
    
        # Calculate success rate per subject
        list_sr_100 = []
        list_sr_99_70 = []
        list_sr_69_1 = []
        list_sr_0 = []

        print('')
        print('#'*60)
        print(f'Background: {bg}, Scale: {scale_percent}, Max Frames: {max_frames}')
        print(algorithm, '-', file_name)
        print('Total:', total)
        print('Success:', success)
        print('Fail:', fail)
        print('SR%:', success_rate, '%')
        print('')

        for subject in list_subject_testing:

            df_results_subject = df_results[
                (df_results['Algorithm'] == algorithm) &
                (df_results['Subject'] == subject) &
                (df_results['File_Name'] == file_name)]
            
            list_images = df_results_subject['Image Data'].tolist()
            list_volume = df_results_subject['Volume'].tolist()
            list_volume_zero = [x for x in list_volume if x == 0]

            subject_total = len(list_volume)
            subject_fail = len(list_volume_zero)
            subject_success = subject_total - subject_fail
            try:
                subject_success_rate = subject_success/subject_total*100
            except:
                subject_success_rate = 0
            
            if subject_success_rate == 100:
                list_sr_100.append(subject)
            elif (subject_success_rate <= 99) and (subject_success_rate >= 70):
                list_sr_99_70.append(subject)
            elif (subject_success_rate <= 69) and (subject_success_rate >= 1):
                list_sr_69_1.append(subject)
            else:
                list_sr_0.append(subject)
                
            # print(f'Subject {subject}: {subject_success_rate}')
            if subject_success_rate == 100:
                print(f'Subject {subject}: {list_images}')

        print('')
        print(f'100%  : {list_sr_100} - {len(list_sr_100)}')
        print(f'99-70%: {list_sr_99_70} - {len(list_sr_99_70)}')
        print(f'69-1% : {list_sr_69_1} - {len(list_sr_69_1)}')
        print(f'0%    : {list_sr_0} - {len(list_sr_0)}')

        list_sr_100 = ', '.join(list_sr_100)
        list_sr_99_70 = ', '.join(list_sr_99_70)
        list_sr_69_1 = ', '.join(list_sr_69_1)
        list_sr_0 = ', '.join(list_sr_0)

        results = [sfm_model, algorithm, bg, scale_percent, max_frames, file_name, 
                    total, success, fail, success_rate,
                    list_sr_100, list_sr_99_70, list_sr_69_1, list_sr_0]
        df_results_save.append(results)
    
columns = ['SFM', 'Algorithm', 'Background', 'Scale', 'Max Frame', 'File',
           'Total', 'Success', 'Fail', 'Success Rate',
           '100', '99-70', '69-1', '0']
df_results_save = pd.DataFrame(df_results_save, columns=columns)
df_results_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume_summary.csv', index=False)

In [ ]:
df_results_save

In [ ]:
algorithm

In [69]:
list_subject_testing = list(set(df_results['Subject'].tolist()))
list_subject_testing = [int(x) for x in list_subject_testing]
list_subject_testing.sort()
list_subject_testing = [str(x) for x in list_subject_testing]

list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']
list_sfm_model = ['MVG_MVS', 'COLAMP_MVS']

df_results_save = []

for algorithm in list_algorithm:
    
    if algorithm in ['MVG', 'MVG_MVS']:
        sfm_model = 'MVG_MVS'
    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
        sfm_model = 'COLMAP_MVS'
    else:
        sfm_model = algorithm
    
    if algorithm in ['MVG', 'COLMAP']:
        list_file_name = ['cloud_and_poses.ply']
    elif algorithm in ['MVG_MVS', 'COLMAP_MVS']:
        list_file_name = ['scene_dense_texture.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_mesh.ply']
        
    for file_name in list_file_name:
        
        for subject in list_subject_testing:

            df_results_subject = df_results[(df_results['Subject'] == subject)]    
            df_results_subject = df_results_subject[
                                                    (df_results_subject['Algorithm'] == algorithm) &
                                                    (df_results_subject['BG'] == 'with_bg') &
                                                    (df_results_subject['Scale'] == 50) &
                                                    (df_results_subject['Max Frames'] == 3) &
                                                    (df_results_subject['File_Name'] == file_name)]    
            
            list_images = df_results_subject['Image Data'].tolist()
            list_volume = df_results_subject['Volume'].tolist()
            list_volume_zero = [x for x in list_volume if x == 0]
            
            total = len(list_volume)
            fail = len(list_volume_zero)
            success = total - fail
            
            try:
                success_rate = success/total*100
            except:
                success_rate = 0 
            
            results = [algorithm, sfm_model, file_name, subject, ', '.join(list_images), total, fail, success, success_rate]
            df_results_save.append(results)
            
            print(f'Subject {subject}: {success_rate}')

columns = ['Algorithm', 'SFM', 'File_Name', 'Subject', 'Image Data', 'Total', 'Fail', 'Success', 'Success_rate']
df_results_save = pd.DataFrame(df_results_save, columns=columns)
df_results_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_SFM_subject_success_rate.csv', index=False)

def print_specific_data(df, title):
    print(f'{title}: {len(df)} -> {", ".join(df['Subject'].tolist())}')
    return None


for algorithm in list_algorithm:
    print('')
    print(algorithm)
    df_temp = df_results_save[df_results_save['Algorithm'] == algorithm]
    if algorithm in ['MVG_MVS', 'COLMAP_MVS']:
        df_temp = df_results_save[df_results_save['File_Name'] == 'scene_dense_mesh.ply']
    print_specific_data(df_temp[(df_temp['Success_rate'] == 100)], '100%')
    print_specific_data(df_temp[(df_temp['Success_rate'] < 100) & (df_temp['Success_rate'] >= 70)], '99-70%')
    print_specific_data(df_temp[(df_temp['Success_rate'] < 70) & (df_temp['Success_rate'] >= 50)], '69-50%')
    print_specific_data(df_temp[(df_temp['Success_rate'] <= 49) & (df_temp['Success_rate'] > 0)], '49-1%')
    print_specific_data(df_temp[(df_temp['Success_rate'] == 0)], '0%')

Subject 1: 100.0
Subject 2: 100.0
Subject 3: 75.0
Subject 4: 83.33333333333334
Subject 5: 62.5
Subject 6: 100.0
Subject 7: 100.0
Subject 8: 100.0
Subject 9: 100.0
Subject 10: 50.0
Subject 11: 50.0
Subject 12: 50.0
Subject 13: 100.0
Subject 14: 50.0
Subject 15: 100.0
Subject 16: 100.0
Subject 17: 100.0
Subject 18: 100.0
Subject 19: 100.0
Subject 20: 75.0
Subject 21: 50.0
Subject 22: 85.71428571428571
Subject 23: 75.0
Subject 24: 83.33333333333334
Subject 25: 87.5
Subject 26: 75.0
Subject 27: 75.0
Subject 28: 75.0
Subject 29: 100.0
Subject 30: 75.0
Subject 31: 30.0
Subject 32: 83.33333333333334
Subject 33: 20.0
Subject 34: 50.0
Subject 35: 83.33333333333334
Subject 36: 75.0
Subject 37: 50.0
Subject 38: 50.0
Subject 39: 62.5
Subject 40: 0.0
Subject 41: 50.0
Subject 42: 83.33333333333334
Subject 43: 90.0
Subject 44: 50.0
Subject 45: 100.0
Subject 46: 100.0
Subject 47: 90.0
Subject 48: 37.5
Subject 49: 100.0
Subject 50: 70.0
Subject 51: 100.0
Subject 52: 50.0
Subject 53: 0.0
Subject 54: 75.

In [66]:
df_results_save

,Algorithm,SFM,Subject,Image Data,Total,Fail,Success,Success_rate
0,MVG,MVG_MVS,1,"1D0L, 1D0R",2,0,2,100.000000
1,MVG,MVG_MVS,2,"2D4L, 2D5R, 2D5L, 2D0R, 2D4R, 2D0L",6,0,6,100.000000
2,MVG,MVG_MVS,3,"3D2R, 3D3L, 3D1R, 3D2L, 3D1L, 3D0L, 3D3R, 3D0R",8,2,6,75.000000
3,MVG,MVG_MVS,4,"4D2R, 4D0L, 4D1L, 4D1R, 4D0R, 4D2L",6,1,5,83.333333
4,MVG,MVG_MVS,5,"5D2L, 5D0R, 5D2R, 5D1L, 5D3R, 5D3L, 5D1R, 5D0L",8,3,5,62.500000
...,...,...,...,...,...,...,...,...
555,COLMAP_MVS,COLMAP_MVS,66,"66D2R, 66D0R, 66D0L, 66D2L, 66D3R, 66D1L, 66D1...",8,4,4,50.000000
556,COLMAP_MVS,COLMAP_MVS,67,"67D2R, 67D2L, 67D0R, 67D0L",4,2,2,50.000000
557,COLMAP_MVS,COLMAP_MVS,68,"68D0L, 68D3R, 68D0R, 68D3L",4,1,3,75.000000
558,COLMAP_MVS,COLMAP_MVS,69,"69D2R, 69D0L, 69D0R, 69D1R, 69D1L, 69D2L",6,1,5,83.333333


,Subject,Image Data,SFM,Algorithm,BG,Scale,Max Frames,Volume Calculation,File_Path,File_Name,Volume,Error
34,17,17D1L,MVG_MVS,MVG,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,13.953985,
35,17,17D1R,MVG_MVS,MVG,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,4.279863,
36,17,17D0L,MVG_MVS,MVG,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,9.246927,
37,17,17D2R,MVG_MVS,MVG,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.665568,
38,17,17D4R,MVG_MVS,MVG,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,40.509789,
...,...,...,...,...,...,...,...,...,...,...,...,...
2227,17,17D0R,COLMAP_MVS,COLMAP_MVS,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_texture.ply,382.905227,
2228,17,17D0R,COLMAP_MVS,COLMAP_MVS,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh_refine.ply,382.906340,
2229,17,17D4L,COLMAP_MVS,COLMAP_MVS,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh.ply,157.619840,
2230,17,17D4L,COLMAP_MVS,COLMAP_MVS,with_bg,50,3,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_texture.ply,156.537236,


In [70]:
list_subject_best = [1, 17, 29, 12, 21, 36, 49, 61]
list_subject_best = [str(x) for x in list_subject_best]

df_filter = df_results[df_results['Subject'].isin(list_subject_best)].reset_index(drop=True)

for i, row in df_filter.iterrows():
    subject = row['Subject']
    data = row['Image Data']
    algorithm = row['Algorithm']
    volume = round(row['Volume'], 3)
    filename = row['File_Name']
    
    print(f'Subject {subject} - {data} - {algorithm} - {filename} - {volume}')

Subject 1 - 1D0L - MVG - cloud_and_poses.ply - 1.498
Subject 1 - 1D0R - MVG - cloud_and_poses.ply - 0.246
Subject 12 - 12D0L - MVG - cloud_and_poses.ply - 0.0
Subject 12 - 12D0R - MVG - cloud_and_poses.ply - 166.889
Subject 17 - 17D1L - MVG - cloud_and_poses.ply - 13.954
Subject 17 - 17D1R - MVG - cloud_and_poses.ply - 4.28
Subject 17 - 17D0L - MVG - cloud_and_poses.ply - 9.247
Subject 17 - 17D2R - MVG - cloud_and_poses.ply - 0.666
Subject 17 - 17D4R - MVG - cloud_and_poses.ply - 40.51
Subject 17 - 17D3L - MVG - cloud_and_poses.ply - 11.492
Subject 17 - 17D3R - MVG - cloud_and_poses.ply - 13.103
Subject 17 - 17D2L - MVG - cloud_and_poses.ply - 11.106
Subject 17 - 17D0R - MVG - cloud_and_poses.ply - 1.504
Subject 17 - 17D4L - MVG - cloud_and_poses.ply - 17.018
Subject 21 - 21D0R - MVG - cloud_and_poses.ply - 4.186
Subject 21 - 21D0L - MVG - cloud_and_poses.ply - 0.0
Subject 29 - 29D0L - MVG - cloud_and_poses.ply - 62.719
Subject 29 - 29D0R - MVG - cloud_and_poses.ply - 406.593
Subject 2

In [ ]:
df_results

In [ ]:
files_ply = [
    ('SFM', 'cloud_and_poses.ply'), 
    ('MVS_mesh', 'scene_dense_mesh.ply'),
    ('MVS_dense', 'scene_dense_texture.ply'),
    ('MVS_refine', 'scene_dense_mesh_refine.ply'),
    ]

df_to_save = []
for subject in ['1', '6', '12', '17', '29', '67']:
    df_results_subject = df_results[df_results['Subject'] == subject].reset_index(drop=True)
    
    list_file_name = list(set(df_results_subject['Image Data'].tolist()))
    list_file_name.sort()

    
    for file_name in list_file_name:
        df_results_subject_file = df_results_subject[df_results_subject['Image Data'] == file_name].reset_index(drop=True)
        
        to_save_list = []

        for mesh_type, ply in files_ply:
            volume = df_results_subject_file[df_results_subject_file['File_Name'] == ply].reset_index(drop=True)['Volume'][0]
            file_path = df_results_subject_file[df_results_subject_file['File_Name'] == ply].reset_index(drop=True)['File_Path'][0]
            
            df_to_save.append([subject, file_name, mesh_type, os.path.join(file_path, ply), volume])
            
columns = ['Subject', 'Image Data', 'Type', 'File_Path', 'Volume']
df_to_save = pd.DataFrame(df_to_save, columns=columns)

df_to_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_BEST_subjects.csv', index=False)
    
    

In [ ]:
df_to_save